In [1]:
# First, let's import necessary modules from TensorFlow
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  # To make TensorFlow v1 code compatible with v2

# Assume the model functions are already defined
from model import DecomNet,Illumination_adjust_curve_net_ratio,Restoration_net

# Now let's try to build the model with a random input
# Replace the real input image with a random tensor
training = tf.placeholder_with_default(False, shape=(), name='training')
input_image = tf.random.normal([1, 256, 256, 3])  # Assume the input image size is 256x256x3
input_decom = tf.placeholder(tf.float32, [None, None, None, 3], name='input_decom')
input_low_r = tf.placeholder(tf.float32, [None, None, None, 3], name='input_low_r')
input_low_i = tf.placeholder(tf.float32, [None, None, None, 1], name='input_low_i')
input_low_i_ratio = tf.placeholder(tf.float32, [None, None, None, 1], name='input_low_i_ratio')
# # Call the model building functions
# R_low, I_low = DecomNet(input_image)
# output_i, A = Illumination_adjust_curve_net_ratio(input_low_i, input_low_i_ratio)
# output_r = Restoration_net(input_low_r, input_low_i, training)
# # Get the output nodes' names
# R_low_name = R_low.op.name
# I_low_name = I_low.op.name

def KinD_LCE(input_decom, input_low_r, input_low_i, input_low_i_ratio, training):
    [R_decom, I_decom] = DecomNet(input_decom)
    decom_output_R = R_decom
    decom_output_I = I_decom
    output_i, A = Illumination_adjust_curve_net_ratio(input_low_i, input_low_i_ratio)
    output_r = Restoration_net(input_low_r, input_low_i, training)
    
    return output_i, output_r, decom_output_R, decom_output_I
output_i, output_r, decom_output_R, decom_output_I = KinD_LCE(input_decom, input_low_r, input_low_i, input_low_i_ratio, training)

# print(f"layer names:{R_low_name}, {I_low_name},{output_i.op.name},{output_r.op.name}")
print(f"kind_layer:{output_i.op.name},{output_r.op.name},{decom_output_R.op.name},{decom_output_I.op.name}")

Instructions for updating:
non-resource variables are not supported in the long term


/home/nvidia/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Instructions for updating:
Colocations handled automatically by placer.


/home/nvidia/Desktop/PommesPeter/lowlight-enhancement-research/KinD_plus/msia_BN_3_M.py:13: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  concat = tf.layers.conv2d(input_i,
/home/nvidia/Desktop/PommesPeter/lowlight-enhancement-research/KinD_plus/msia_BN_3_M.py:33: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  pu_conv = tf.layers.batch_normalization(pu_conv, training=training)
/home/nvidia/Desktop/PommesPeter/lowlight-enhancement-research/KinD_plus/msia_BN_3_M.py:40: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.c

kind_layer:I_enhance_Net_ratio/Sigmoid,Denoise_Net/Sigmoid,DecomNet/Sigmoid,DecomNet/Sigmoid_1


In [21]:
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
import os

# Assume the model functions are already defined
from model import DecomNet, Illumination_adjust_curve_net_ratio, Restoration_net

def KinD_LCE(input_decom, input_low_r, input_low_i, input_low_i_ratio, training):
    [R_decom, I_decom] = DecomNet(input_decom)
    decom_output_R = R_decom
    decom_output_I = I_decom
    output_i, A = Illumination_adjust_curve_net_ratio(input_low_i, input_low_i_ratio)
    output_r = Restoration_net(input_low_r, input_low_i, training)
    
    return output_i, output_r, decom_output_R, decom_output_I

# Create a session and make it the default session
sess = tf.Session()
tf.keras.backend.set_session(sess)

# Build the model
input_decom = tf.placeholder(tf.float32, [None, None, None, 3], name='input_decom')
input_low_r = tf.placeholder(tf.float32, [None, None, None, 3], name='input_low_r')
input_low_i = tf.placeholder(tf.float32, [None, None, None, 1], name='input_low_i')
input_low_i_ratio = tf.placeholder(tf.float32, [None, None, None, 1], name='input_low_i_ratio')
training = tf.placeholder_with_default(False, shape=(), name='training')

# Call the model building function
output_i, output_r, decom_output_R, decom_output_I = KinD_LCE(input_decom, input_low_r, input_low_i, input_low_i_ratio, training)

# Assume you have trained the model and the weights are loaded

# We will save the model in the Protobuf format
print(f"outputname:{output_i.op.name}")
print(f"outputname:{output_r.op.name}")
print(f"outputname:{decom_output_R.op.name}")
print(f"outputname:{decom_output_I.op.name}")
output_node_names = ['I_enhance_Net_ratio/Sigmoid', 'Denoise_Net/Sigmoid', 'DecomNet/Sigmoid', 'DecomNet/Sigmoid_1']
input_node_names = ['input_decom', 'input_low_r', 'input_low_i', 'input_low_i_ratio', 'training']
output_graph_name = './model.pb'
checkpoint_dir = './experiment/exp2/checkpoint'
illmin_name = "illumination_adjust_curve_net_global_rm_del_rotate"

var_Decom = [var for var in tf.trainable_variables() if 'DecomNet' in var.name]
var_adjust = [var for var in tf.trainable_variables() if 'I_enhance_Net' in var.name]
var_restoration = [var for var in tf.trainable_variables() if 'Denoise_Net' in var.name]
g_list = tf.global_variables()
bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
var_restoration += bn_moving_vars
saver_Decom = tf.train.Saver(var_list=var_Decom)
saver_adjust = tf.train.Saver(var_list=var_adjust)
saver_restoration = tf.train.Saver(var_list=var_restoration)
decom_checkpoint_dir = os.path.join(checkpoint_dir, 'decom_net_retrain')
ckpt_pre = tf.train.get_checkpoint_state(decom_checkpoint_dir)
if ckpt_pre:
    print('[*] loaded ' + ckpt_pre.model_checkpoint_path)
    saver_Decom.restore(sess, ckpt_pre.model_checkpoint_path)
else:
    print('[*] No decomnet pretrained model!')

checkpoint_dir_adjust = os.path.join(checkpoint_dir, illmin_name)
ckpt_adjust = tf.train.get_checkpoint_state(checkpoint_dir_adjust)
if ckpt_adjust:
    print('[*] loaded ' + ckpt_adjust.model_checkpoint_path)
    saver_adjust.restore(sess, ckpt_adjust.model_checkpoint_path)
else:
    print("[*] No adjust net pretrained model!")

checkpoint_dir_restoration = os.path.join(checkpoint_dir, 'new_restoration_retrain')
ckpt = tf.train.get_checkpoint_state(checkpoint_dir_restoration)
if ckpt:
    print('[*] loaded ' + ckpt.model_checkpoint_path)
    saver_restoration.restore(sess, ckpt.model_checkpoint_path)
else:
    print("[*] No restoration net pretrained model!")


# Convert variables to constants and save the model
output_graph_def = tf.graph_util.convert_variables_to_constants(
    sess,  # The session is used to retrieve the weights
    tf.get_default_graph().as_graph_def(),  # The graph_def is used to retrieve the nodes
    output_node_names  # The output node names are used to define the usefull nodes
)

# Finally we serialize and dump the output graph to the filesystem
with tf.gfile.GFile(output_graph_name, "wb") as f:
    f.write(output_graph_def.SerializeToString())

print("%d ops in the final graph." % len(output_graph_def.node))

2023-08-04 19:32:52.740166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 19:32:52.740644: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 19:32:52.741056: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 19:32:52.741584: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 19:32:52.741743: I tensorflow/core/co

/home/nvidia/Desktop/PommesPeter/lowlight-enhancement-research/KinD_plus/msia_BN_3_M.py:13: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  concat = tf.layers.conv2d(input_i,
/home/nvidia/Desktop/PommesPeter/lowlight-enhancement-research/KinD_plus/msia_BN_3_M.py:33: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  pu_conv = tf.layers.batch_normalization(pu_conv, training=training)
/home/nvidia/Desktop/PommesPeter/lowlight-enhancement-research/KinD_plus/msia_BN_3_M.py:40: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.c

outputname:I_enhance_Net_ratio_5/Sigmoid
outputname:Denoise_Net_5/Sigmoid
outputname:DecomNet_5/Sigmoid
outputname:DecomNet_5/Sigmoid_1
[*] loaded ./experiment/exp2/checkpoint/decom_net_retrain/Decomposition_Net.ckpt-150000
INFO:tensorflow:Restoring parameters from ./experiment/exp2/checkpoint/decom_net_retrain/Decomposition_Net.ckpt-150000
[*] loaded ./experiment/exp2/checkpoint/illumination_adjust_curve_net_global_rm_del_rotate/Illumination_Adjustment_Net.ckpt-112000
INFO:tensorflow:Restoring parameters from ./experiment/exp2/checkpoint/illumination_adjust_curve_net_global_rm_del_rotate/Illumination_Adjustment_Net.ckpt-112000
[*] loaded ./experiment/exp2/checkpoint/new_restoration_retrain/Reflectance_Restoration_Net.ckpt-2399
INFO:tensorflow:Restoring parameters from ./experiment/exp2/checkpoint/new_restoration_retrain/Reflectance_Restoration_Net.ckpt-2399
1023 ops in the final graph.


In [30]:
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph

# Assume the model functions are already defined
from model import DecomNet, Illumination_adjust_curve_net_ratio, Restoration_net

# Load the .pb file
with tf.gfile.GFile('model.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

# Import the graph_def into a new Graph
with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def)

# Export the graph as a SavedModel
with tf.Session(graph=graph) as sess:
    tf.saved_model.simple_save(sess,
                               'saved_model_dir',
                               inputs={'input': graph.get_tensor_by_name('input:0')},
                               outputs={'output': graph.get_tensor_by_name('output:0')})



2023-08-04 19:37:51.577150: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 19:37:51.577809: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 19:37:51.578328: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 19:37:51.578890: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 19:37:51.579082: I tensorflow/core/co

KeyError: "The name 'input:0' refers to a Tensor which does not exist. The operation, 'input', does not exist in the graph."

In [31]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt

conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(
    max_workspace_size_bytes=(1<<32),
    precision_mode="FP16",
    maximum_cached_engines=100)

# Create a converter
converter = tf.experimental.tensorrt.Converter(
    input_saved_model_dir='path_to_save_trt_model',
    conversion_params=conversion_params)

# Convert the model
converter.convert()

# Save the optimized model
converter.save('optimized_model.pb')

AssertionError: 